In [1]:

# ! pip install agent-framework-azure-ai -U

In [2]:

from agent_framework.azure import AzureAIAgentClient
from agent_framework import ChatAgent,ChatMessage, DataContent, Role, TextContent

from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential

In [3]:

import os
import base64
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
AgentName = "Vision-Agent"
AgentInstructions = "You are my furniture sales consultant, you can find different furniture elements from the pictures and give me a purchase suggestion"

In [6]:
async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):
        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            instructions=AgentInstructions,
            name=AgentName
        )
        chat_client=AzureAIAgentClient(project_client=client, agent_id=created_agent.id)

        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client
            ) as agent:
    

            image_path = "../../files/home.png"
            with open(image_path, "rb") as image_file:
                image_b64 = base64.b64encode(image_file.read()).decode()
            image_uri = f"data:image/png;base64,{image_b64}"
            message = ChatMessage(
                    role=Role.USER,
                    contents=[
                        TextContent(text="Please find the relevant furniture according to the image and give the corresponding price for each piece of furniture"),
                        DataContent(uri=image_uri, media_type="image/png")
                    ]
            )
            # response = agent.run_stream(message)
            first_result = await agent.run(message)
            
            print(f"Agent: {first_result.text}")

Agent: Based on the image provided, I have identified the key furniture elements in this living room. Below are purchase suggestions and estimated price ranges for each piece of furniture:

1. **Sofa**  
   - Style: Modern, white fabric couch with cushion accents.  
   - Suggested Price: $500–$1,200  

2. **Coffee Table**  
   - Style: Oval-shaped wooden table with minimalist design.  
   - Suggested Price: $150–$400  

3. **TV Console/Stand**  
   - Style: Long, wooden console with drawers for storage.  
   - Suggested Price: $300–$800  

4. **Accent Chair**  
   - Style: Dark blue upholstered chair with high back.  
   - Suggested Price: $250–$600  

5. **Side Table (next to accent chair)**  
   - Style: Round, metallic tripod base with glass top.  
   - Suggested Price: $100–$250  

6. **Pendant Lighting Fixtures**  
   - Style: Modern dangling spherical glass pendant lights.  
   - Suggested Price: $120–$300 per unit  

7. **Chandelier**  
   - Style: Modern black geometric chandel